In [1]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
import time

In [2]:
# Initialize the Earth Engine API
ee.Initialize()

In [4]:

# Define parameters
start_date = '2023-01-01'
end_date = '2023-05-31'
chunk_size_days = 30  # Each chunk is 30 days long

# Load the shapefile and convert to a feature collection
# roi_fc = geemap.shp_to_ee('../TBP Shape Files/Gross Command Area.shp')
roi_fc = ee.FeatureCollection('users/skhan7/PhD/Chapter2/BWDB_Commad_Area_Simplified_Modified2')
field_capacity = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b10').divide(ee.Number(100)).clip(roi_fc)
# Function to upscale soil moisture to 250m resolution
def upscale_to_250m(image,region):
    return image.reduceResolution(
        reducer=ee.Reducer.mean(),
        bestEffort=True,
        geometry = region.geometry()
    ).reproject(crs=image.projection(), scale=250)

# Function to calculate soil moisture for each image within a given region
def calculate_soil_moisture(image, wet_index, dry_index, urban_mask, water_mask, region):
    sensitivity = wet_index.subtract(dry_index)
    Mv = image.select('VV').subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())
    Mv_upscaled = Mv.reduceResolution(
        reducer=ee.Reducer.mean(),
        bestEffort=True,
    ).reproject(crs=Mv.projection(), scale=250)
    # Calculate mean soil moisture for each image in the specified region
    Mv_upscaled_clamped = Mv_upscaled.clamp(0, 0.6)
    mean_ssm = Mv_upscaled_clamped.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region.geometry(),
        scale=250,
        maxPixels=1e9
    ).get('VV')
    percolation = Mv_upscaled_clamped.subtract(field_capacity).max(0)
    mean_percolation = percolation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region.geometry(),
        scale=250,
        maxPixels=1e9
    ).get('VV')
    mean_fieldCapacity= field_capacity.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region.geometry(),
        scale=250,
        maxPixels=1e9
    ).get('b10')

    # Get the date of the image
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    
    return ee.Feature(None, {'date': date, 'MeanSoilMoisture': mean_ssm, 'MeanPercolation': mean_percolation, 'MeanFieldCapacity': mean_fieldCapacity,'RegionID': region.get('CNLNM_ID')})

# Generate a list of start and end dates for each 30-day chunk
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
dates = pd.date_range(start=start_date_dt, end=end_date_dt, freq=f'{chunk_size_days}D')

# Initialize a list to store dataframes for each region
region_dataframes = []

# Iterate over each region (feature) in the ROI feature collection
for region in roi_fc.toList(roi_fc.size()).getInfo():
    region_feature = ee.Feature(region)
    region_id = region_feature.get('CNLNM_ID').getInfo()  # Use 'CNLNMID' as the region ID

    # Process each 30-day chunk for the current region
    all_results = []
    for i in range(len(dates) - 1):
        chunk_start = dates[i].strftime('%Y-%m-%d')
        chunk_end = dates[i + 1].strftime('%Y-%m-%d')
        
        # Load Sentinel-1 Image Collection for the current chunk
        s1_collection_chunk = (ee.ImageCollection('COPERNICUS/S1_GRD')
                               .filterBounds(region_feature.geometry())
                               .filterDate(chunk_start, chunk_end)
                               .filter(ee.Filter.eq('instrumentMode', 'IW'))
                               .select(['VV']))
        
        # Calculate wet and dry indices for the current chunk
        wet_index = s1_collection_chunk.max().select('VV')
        dry_index = s1_collection_chunk.min().select('VV')
        
        # Define urban and water masks
        avg_vv = s1_collection_chunk.mean().select('VV')
        urban_mask = avg_vv.gt(-6)  # Urban: VV > -6 dB
        water_mask = avg_vv.lt(-17)  # Water: VV < -17 dB
        
        # Calculate SSM for each image in the chunk for the current region
        soil_moisture_features = s1_collection_chunk.map(
            lambda image: calculate_soil_moisture(image, wet_index, dry_index, urban_mask, water_mask, region_feature)
        )
        
        # Retrieve SSM values from each image in the chunk
        ssm_features = soil_moisture_features.getInfo()
        chunk_data = [{'Date': f['properties']['date'],
                       'MeanSoilMoisture': f['properties']['MeanSoilMoisture'],
                       'MeanPercolation': f['properties']['MeanPercolation'],
                       'MeanFieldCapacity': f['properties']['MeanFieldCapacity'],
                       'RegionID': f['properties']['RegionID']} for f in ssm_features['features']]
        
        # Append chunk data to all results for the region
        all_results.extend(chunk_data)
    
    # Create a DataFrame for the current region and add it to the list
    region_df = pd.DataFrame(all_results)
    # region_df['MaxSoilMoisture'] = region_df['MeanSoilMoisture'].max()
    region_df['RegionID'] = region_id  # Ensure RegionID is present in DataFrame
    # region_df['NormalizedSoilMoisture'] = (region_df['MeanSoilMoisture'].max() - region_df['MeanSoilMoisture'])/region_df['MeanSoilMoisture'].max()
    region_dataframes.append(region_df)

# Combine all regional dataframes into a single GeoDataFrame
final_df = pd.concat(region_dataframes, ignore_index=True)
# final_df['MaxSoilMoisture'] = final_df.groupby('RegionID')['MeanSoilMoisture'].transform('max')

# Normalize the MeanSoilMoisture for each region
# final_df['NormalizedSoilMoisture'] = (final_df['MaxSoilMoisture'] - final_df['MeanSoilMoisture']) / final_df['MaxSoilMoisture']
# final_df['NormalizedSoilMoisture'] = (final_df['MeanSoilMoisture']) / final_df['MaxSoilMoisture']
# final_df['MeanSoilMoisture'] = final_df['MeanSoilMoisture'].clip(upper=0.5)
# Drop the intermediate MaxSoilMoisture column if you don’t need it
# final_df = final_df.drop(columns=['MaxSoilMoisture'])
print(final_df)


            Date  MeanSoilMoisture  MeanPercolation  MeanFieldCapacity  \
0     2023-01-04          0.304932         0.024472           0.313823   
1     2023-01-07          0.577484         0.265435           0.313823   
2     2023-01-11          0.583333         0.270504           0.313823   
3     2023-01-16          0.290589         0.019735           0.313823   
4     2023-01-19          0.575083         0.261556           0.313823   
...          ...               ...              ...                ...   
5705  2023-05-11          0.416869         0.112277           0.303802   
5706  2023-05-16          0.205611         0.008822           0.303802   
5707  2023-05-19          0.600000         0.294907           0.303802   
5708  2023-05-23          0.600000         0.295463           0.303802   
5709  2023-05-28          0.466889         0.161585           0.303802   

               RegionID  
0              T1S1B_95  
1              T1S1B_95  
2              T1S1B_95  
3      

In [14]:
# Set the date range for the last 7 days
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')

# Load the shapefile as a feature collection
roi_fc = ee.FeatureCollection('users/skhan7/PhD/Chapter2/BWDB_Commad_Area_Simplified_Modified2')
field_capacity = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b10').divide(ee.Number(100)).clip(roi_fc)

# Initialize a list to store dataframes for each region
region_dataframes = []

# Iterate over each region (feature) in the ROI feature collection
for region in roi_fc.toList(roi_fc.size()).getInfo():
    region_feature = ee.Feature(region)
    region_id = region_feature.get('CNLNM_ID').getInfo()
    region_geometry = region_feature.geometry()

    # Load Sentinel-1 Image Collection for the last 7 days and apply speckle filtering
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(region_geometry)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))

    # Apply smoothing, calculate soil moisture, and create a feature collection for the output
    def process_image(image):
        # Apply focal median filter for smoothing
        smoothed = image.addBands(image.focal_max(30, 'circle', 'meters').rename("Smooth"))

        # Calculate wet and dry indices using smoothed collection
        wet_index = s1_collection.max().select('VV')
        dry_index = s1_collection.min().select('VV')
        sensitivity = wet_index.subtract(dry_index)

        # Define urban and water masks
        avg_vv = smoothed.select('Smooth').reduceRegion(reducer=ee.Reducer.mean(), geometry=region_geometry, scale=250)
        urban_mask = smoothed.select('Smooth').gt(-6)
        water_mask = smoothed.select('Smooth').lt(-17)

        # Calculate soil moisture
        Mv = smoothed.select("Smooth").subtract(dry_index).divide(sensitivity)
        Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())

        # Upscale and clamp soil moisture
        Mv_upscaled = Mv.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=Mv.projection(), scale=250)
        Mv_upscaled_clamped = Mv_upscaled.clamp(0, 0.6)

        # Calculate mean soil moisture in the region
        mean_ssm = Mv_upscaled_clamped.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=region_geometry,
            scale=250,
            maxPixels=1e9
        ).get('Smooth')

        # Calculate field capacity
        mean_field_capacity = field_capacity.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=region_geometry,
            scale=250,
            maxPixels=1e9
        ).get('b10')

        # Calculate percolation
        percolation = Mv_upscaled_clamped.subtract(field_capacity).max(0)
        mean_percolation = percolation.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=region_geometry,
            scale=250,
            maxPixels=1e9
        ).get('Smooth')

        
        # Get the date of the image
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        
        # Return as a feature
        return ee.Feature(None, {
            'date': date,
            'MeanSoilMoisture': mean_ssm,
            'MeanPercolation': mean_percolation,
            'MeanFieldCapacity': mean_field_capacity,
            'CNLNM_ID': region_id
        })

    # Apply the processing function to each image in the smoothed collection
    soil_moisture_features = s1_collection.map(process_image)

    # Retrieve SSM, percolation, and field capacity values and create a DataFrame for each region
    ssm_features = soil_moisture_features.getInfo()
    week_data = [{'Date': f['properties']['date'],
                  'MeanSoilMoisture': f['properties']['MeanSoilMoisture'],
                  'MeanPercolation': f['properties']['MeanPercolation'],
                  'MeanFieldCapacity': f['properties']['MeanFieldCapacity'],
                  'CNLNM_ID': f['properties']['CNLNM_ID']} for f in ssm_features['features']]
    
    # Calculate the mean soil moisture over the last 7 days for each region
    region_df = pd.DataFrame(week_data)
    mean_ssm_week = region_df['MeanSoilMoisture'].astype(float).mean()
    region_df['MeanSoilMoistureWeekly'] = mean_ssm_week
    region_dataframes.append(region_df)

# Combine all regional dataframes into a single DataFrame
final_df = pd.concat(region_dataframes, ignore_index=True)

# Visualize the images on the map to inspect them
Map = geemap.Map()
Map.centerObject(roi_fc, 8)

# Display first original and first smoothed images for visual inspection
first_image = ee.Image(s1_collection.first())
smoothed_first_image = first_image.addBands(first_image.focal_max(30, 'circle', 'meters').rename("Smooth"))

Map.addLayer(first_image, {'min': -25, 'max': 0}, "Original VV")
Map.addLayer(smoothed_first_image.select("Smooth"), {'min': -25, 'max': 0}, "Smoothed VV")
Map

# Print the final DataFrame
print(final_df)


           Date  MeanSoilMoisture  MeanPercolation  MeanFieldCapacity  \
0    2024-11-06            0.0000              0.0               0.31   
1    2024-11-06            0.0000              0.0               0.31   
2    2024-11-06            0.0000              0.0               0.31   
3    2024-11-06            0.0000              0.0               0.32   
4    2024-11-06            0.0000              0.0               0.32   
..          ...               ...              ...                ...   
190  2024-11-03            0.0000              0.0               0.32   
191  2024-11-06            0.2176              0.0               0.32   
192  2024-11-06            0.0000              0.0               0.33   
193  2024-11-06            0.0000              0.0               0.31   
194  2024-11-06            0.0000              0.0               0.30   

              CNLNM_ID  MeanSoilMoistureWeekly  
0             T1S1B_95                  0.0000  
1            T2S1B_101   

In [20]:
from datetime import datetime, timedelta
import pandas as pd
import geemap
import ee
from tqdm import tqdm 
# Initialize Earth Engine API and geemap
ee.Initialize()

# Set the date range for the last 7 days
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')

# Load the shapefile as a feature collection
roi_fc = ee.FeatureCollection('users/skhan7/PhD/Chapter2/BWDB_Commad_Area_Simplified_Modified2')
field_capacity = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b10').divide(ee.Number(100)).clip(roi_fc)

# Initialize a list to store dataframes for each region
region_dataframes = []
projection = ee.ImageCollection("COPERNICUS/S1_GRD").select('VV').filterBounds(roi_fc).first().projection()
# Iterate over each region (feature) in the ROI feature collection
region_list = roi_fc.toList(roi_fc.size()).getInfo()
for region in tqdm(region_list, desc="Soil Mositure Estimation", unit="Command Area"):
    region_feature = ee.Feature(region)
    region_id = region_feature.get('CNLNM_ID').getInfo()
    region_geometry = region_feature.geometry()

    # Load Sentinel-1 Image Collection for the last 7 days and calculate the mean image
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(region_geometry)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))

    # Calculate mean VV for the 7-day period and apply speckle filtering
    mean_image = s1_collection.mean().setDefaultProjection(projection)
    smoothed = mean_image.addBands(mean_image.focal_max(30, 'circle', 'meters').rename("Smooth"))

    # Calculate wet and dry indices using the mean smoothed image
    wet_index = s1_collection.max().select('VV')
    dry_index = s1_collection.min().select('VV')
    sensitivity = wet_index.subtract(dry_index)

    # Define urban and water masks
    urban_mask = smoothed.select('Smooth').gt(-6)
    water_mask = smoothed.select('Smooth').lt(-17)

    # Calculate soil moisture
    Mv = smoothed.select("Smooth").subtract(dry_index).divide(sensitivity)
    Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())

    # Upscale and clamp soil moisture
    Mv_upscaled = Mv.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=Mv.projection(), scale=250)
    Mv_upscaled_clamped = Mv_upscaled.clamp(0, 0.6)

    # Calculate mean soil moisture in the region
    median_ssm = Mv_upscaled_clamped.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=region_geometry,
        scale=250,
        maxPixels=1e9
    ).get('Smooth')

    # Calculate percolation
    percolation = Mv_upscaled_clamped.subtract(field_capacity).max(0)
    median_percolation = percolation.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=region_geometry,
        scale=250,
        maxPixels=1e9
    ).get('Smooth')

    # Calculate field capacity
    median_field_capacity = field_capacity.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=region_geometry,
        scale=250,
        maxPixels=1e9
    ).get('b10')

    # Store results in a dictionary for each region
    region_data = {
        'CNLNM_ID': region_id,
        'MedianSoilMoisture': median_ssm.getInfo() if median_ssm else None,
        'MedianPercolation': median_percolation.getInfo() if median_percolation else None,
        'MedianFieldCapacity': median_field_capacity.getInfo() if median_field_capacity else None
    }
    
    # Append region data to the dataframe list
    region_dataframes.append(region_data)

# Convert the results to a DataFrame
final_df = pd.DataFrame(region_dataframes)



Soil Mositure Estimation:   6%|▌         | 9/153 [05:47<1:32:46, 38.65s/Command Area]


KeyboardInterrupt: 

In [24]:
from datetime import datetime, timedelta
import pandas as pd
import geemap
import ee
from tqdm import tqdm

# Initialize Earth Engine API and geemap
ee.Initialize()

# Set the date range for the last 7 days
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')

# Load the shapefile as a feature collection
roi_fc = ee.FeatureCollection('users/skhan7/PhD/Chapter2/BWDB_Commad_Area_Simplified_Modified2')
field_capacity = ee.Image("OpenLandMap/SOL/SOL_WATERCONTENT-33KPA_USDA-4B1C_M/v01").select('b10').divide(ee.Number(100)).clip(roi_fc)

# Initialize a list to store dataframes for each region
region_dataframes = []

# Use tqdm to add a progress bar for the number of regions
region_list = roi_fc.toList(roi_fc.size()).getInfo()
for region in tqdm(region_list, desc="Estimating Soil Mositure", unit=" Command Area"):
    region_feature = ee.Feature(region)
    region_id = region_feature.get('CNLNM_ID').getInfo()
    region_geometry = region_feature.geometry()

    # Load Sentinel-1 Image Collection for the last 7 days and apply speckle filtering
    s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                     .filterBounds(region_geometry)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
                     .select(['VV']))

    # Apply smoothing, calculate soil moisture, and create a feature collection for the output
    def process_image(image):
        # Apply focal median filter for smoothing
        smoothed = image.addBands(image.focal_max(30, 'circle', 'meters').rename("Smooth"))

        # Calculate wet and dry indices using smoothed collection
        wet_index = s1_collection.max().select('VV')
        dry_index = s1_collection.min().select('VV')
        sensitivity = wet_index.subtract(dry_index)

        # Define urban and water masks
        urban_mask = smoothed.select('Smooth').gt(-6)
        water_mask = smoothed.select('Smooth').lt(-17)

        # Calculate soil moisture
        Mv = smoothed.select("Smooth").subtract(dry_index).divide(sensitivity)
        Mv = Mv.updateMask(water_mask.Not()).updateMask(urban_mask.Not())

        # Upscale and clamp soil moisture
        Mv_upscaled = Mv.reduceResolution(reducer=ee.Reducer.mean(), bestEffort=True).reproject(crs=Mv.projection(), scale=250)
        Mv_upscaled_clamped = Mv_upscaled.clamp(0, 0.6)

        # Calculate mean soil moisture in the region
        median_ssm = Mv_upscaled_clamped.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=region_geometry,
            scale=250,
            maxPixels=1e9
        ).get('Smooth')

        # Calculate field capacity
        median_field_capacity = field_capacity.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=region_geometry,
            scale=250,
            maxPixels=1e9
        ).get('b10')

        # # Calculate percolation
        # percolation = Mv_upscaled_clamped.subtract(field_capacity).max(0)
        # median_percolation = percolation.reduceRegion(
        #     reducer=ee.Reducer.median(),
        #     geometry=region_geometry,
        #     scale=250,
        #     maxPixels=1e9
        # ).get('Smooth')

        # Return as a feature with aggregated data for the period
        return ee.Feature(None, {
            'CNLNM_ID': region_id,
            'MedianSoilMoisture': median_ssm,
            'MedianFieldCapacity': median_field_capacity
        })

    # Apply the processing function to each image in the smoothed collection
    soil_moisture_features = s1_collection.map(process_image)

    # Retrieve data for each region and create a DataFrame
    ssm_features = soil_moisture_features.getInfo()
    week_data = [{'CNLNM_ID': f['properties']['CNLNM_ID'],
                  'MedianSoilMoisture': f['properties']['MedianSoilMoisture'],
                  'MedianFieldCapacity': f['properties']['MedianFieldCapacity']}
                 for f in ssm_features['features']]
    
    # Append the data for the current region to the list
    region_df = pd.DataFrame(week_data)
    region_dataframes.append(region_df)

# Combine all regional dataframes into a single DataFrame
final_df = pd.concat(region_dataframes, ignore_index=True)
# Group by region and calculate the mean values for each metric
final_means_df = final_df.groupby('CNLNM_ID').mean().reset_index()
final_means_df['MedianPercolation'] = (final_means_df['MedianSoilMoisture'] - final_means_df['MedianFieldCapacity']).clip(lower=0)
# Print the final DataFrame with mean values for each region
print(final_means_df)


Estimating Soil Mositure: 100%|██████████| 153/153 [01:08<00:00,  2.22 Command Area/s]

               CNLNM_ID  MedianSoilMoisture  MedianFieldCapacity  \
0       Bogra_Canal_104                 0.3                 0.28   
1        Bogra_Canal_78                 0.0                 0.30   
2        Bogra_Canal_92                 0.0                 0.31   
3    Dinajpur_Canal_123                 0.0                 0.28   
4     Dinajpur_Canal_21                 0.0                 0.33   
..                  ...                 ...                  ...   
148   Teesta_Canal_A_13                 0.0                 0.32   
149   Teesta_Canal_A_16                 0.0                 0.32   
150    Teesta_Canal_A_4                 0.0                 0.33   
151   Teesta_Canal_B_18                 0.0                 0.32   
152   Teesta_Canal_B_25                 0.0                 0.33   

     MedianPercolation  
0                 0.02  
1                 0.00  
2                 0.00  
3                 0.00  
4                 0.00  
..                 ...  
148     

In [ ]:

# Print the final DataFrame
print(final_df)